### Machine Learning algorithms and Diabete

In [2]:
from pyspark.sql.functions import *
from pyspark.sql import functions as fn
from pyspark.sql.functions import mean, col, stddev
from pyspark.mllib.util import MLUtils
from pyspark.mllib.stat import Statistics
import pyspark.mllib.linalg
from pyspark.mllib.linalg import Matrix, Matrices
from pyspark.mllib.linalg.distributed import *
from pyspark.mllib.linalg import Vectors
from pyspark.ml.stat import Correlation
from pyspark.mllib.stat import KernelDensity
from pyspark.mllib.linalg import Matrices, Vectors
from pyspark.mllib.regression import LabeledPoint
from numpy import array
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.linalg import DenseVector
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel


import pandas as pd
import matplotlib.pyplot as plt
import pylab
import seaborn as sns
from scipy.stats import norm
import scipy.stats as stats
import numpy as np
import statsmodels.api as sm
from string import ascii_letters
import scipy.stats as ss

from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import PCA, VectorAssembler, StringIndexer
from pyspark.ml.feature import Normalizer

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel, LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline


from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn import model_selection
from sklearn.ensemble import  GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [3]:
!wget -O diabete.csv https://raw.githubusercontent.com/susanli2016/Machine-Learning-with-Python/master/diabetes.csv
df = pd.read_csv('diabete.csv') # panda dataframe
spark_df = sqlContext.createDataFrame(df) # spark data frame

In [4]:
spark_df.printSchema()

root
-- Pregnancies: long (nullable = true)
-- Glucose: long (nullable = true)
-- BloodPressure: long (nullable = true)
-- SkinThickness: long (nullable = true)
-- Insulin: long (nullable = true)
-- BMI: double (nullable = true)
-- DiabetesPedigreeFunction: double (nullable = true)
-- Age: long (nullable = true)
-- Outcome: long (nullable = true)

In [5]:
display(spark_df)

Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
6,148,72,35,0,33.6,0.627,50,1
1,85,66,29,0,26.6,0.35100000000000003,31,0
8,183,64,0,0,23.3,0.672,32,1
1,89,66,23,94,28.1,0.16699999999999998,21,0
0,137,40,35,168,43.1,2.2880000000000003,33,1
5,116,74,0,0,25.6,0.201,30,0
3,78,50,32,88,31.0,0.248,26,1
10,115,0,0,0,35.3,0.134,29,0
2,197,70,45,543,30.5,0.158,53,1
8,125,96,0,0,0.0,0.23199999999999998,54,1


In [6]:
spark_df.head()

Out[13]: Row(Pregnancies=6, Glucose=148, BloodPressure=72, SkinThickness=35, Insulin=0, BMI=33.6, DiabetesPedigreeFunction=0.627, Age=50, Outcome=1)

In [7]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Getting dimension of dataframe

In [9]:
print("dimension of diabetes data: {}".format(df.shape))

dimension of diabetes data: (768, 9)

Outcome has two values: 0 (No diabete) and 1 (Diabete). We will predict for Outcome. print(diabetes.groupby('Outcome').size())

In [11]:
print(df.groupby('Outcome').size())

Outcome
0 500
1 268
dtype: int64

In [12]:
import seaborn as sns
sns.countplot(df['Outcome'],label="Count")

Out[4]: <matplotlib.axes._subplots.AxesSubplot at 0x7ff00a18a9b0>

Calculating proportion of Diabete

In [14]:
res_target = (spark_df.groupBy("Outcome").\
              agg(fn.count('Outcome').alias('Headcounts'),
            ((fn.count('Outcome')/int(spark_df.count()))*100).alias('Proportions')))

res_target = res_target.withColumn("Proportions", fn.round(res_target["Proportions"], 2))
res_target.registerTempTable("res_target")

display(sqlContext.sql("SELECT * FROM res_target"))

Outcome,Headcounts,Proportions
0,500,65.1
1,268,34.9


Plot the proportion of Diabete and Non-diabete

In [16]:
labels = 'non-diabete', 'diabete'
sizes = [res_target.toPandas()['Headcounts'][0], res_target.toPandas()['Headcounts'][1]]
colors = ['indianred', 'tomato']
explode = (0, 0.1) 
 
# Plot
fig, ax = plt.subplots()
ax = plt.pie(sizes, explode = explode, colors = colors, 
             autopct = '%1.1f%%', shadow = True, startangle = 140)
plt.axis('equal')
plt.title('Repartition of the diabete and non-diabete patients')
plt.legend(labels)

display(fig)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
Pregnancies 768 non-null int64
Glucose 768 non-null int64
BloodPressure 768 non-null int64
SkinThickness 768 non-null int64
Insulin 768 non-null int64
BMI 768 non-null float64
DiabetesPedigreeFunction 768 non-null float64
Age 768 non-null int64
Outcome 768 non-null int64
dtypes: float64(2), int64(7)
memory usage: 54.1 KB

All variables in our data are numeric

##### Preparing for training and test dataset

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'Outcome'], df['Outcome'], stratify=df['Outcome'], random_state=66)

# Logistic regression

In [22]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression().fit(X_train, y_train)

/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
 FutureWarning)

In [23]:
print("Training set accuracy: {:.3f}".format(LR.score(X_train, y_train)))
print("Test set accuracy: {:.3f}".format(LR.score(X_test, y_test)))

Training set accuracy: 0.781
Test set accuracy: 0.771

#### Changing C value to see how the accuracy changC

In [25]:
LR001 = LogisticRegression(C = 0.01).fit(X_train, y_train)
print("Training set accuracy: {:.3f}".format(LR001.score(X_train, y_train)))
print("Test set accuracy: {:.3f}".format(LR001.score(X_test, y_test)))

/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
 FutureWarning)
Training set accuracy: 0.700
Test set accuracy: 0.703

In [26]:
LR100 = LogisticRegression(C = 100).fit(X_train, y_train)
print("Training set accuracy: {:.3f}".format(LR100.score(X_train, y_train)))
print("Test set accuracy: {:.3f}".format(LR100.score(X_test, y_test)))

/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
 FutureWarning)
Training set accuracy: 0.785
Test set accuracy: 0.766

#### We get that they are not so different when C changes. Thus, we use the default value of C = 1

#### Visualization of coefficients for three regularization C = 1, C = 0.01, C = 100

In [29]:
diabetes_features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
plt.figure(figsize=(8,6))
plt.plot(LR.coef_.T, 'o', label="C=1")
plt.plot(LR100.coef_.T, '^', label="C=100")
plt.plot(LR001.coef_.T, 'v', label="C=0.01")
plt.xticks(range(df.shape[1]), diabetes_features, rotation=90)
plt.hlines(0, 0, df.shape[1])
plt.ylim(-5, 5)
plt.xlabel("Feature")
plt.ylabel("Coefficient magnitude")
plt.legend()
#plt.savefig('log_coef')
plt.show()

# Decision tree

In [31]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=0)
tree.fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test, y_test)))

Accuracy on training set: 1.000
Accuracy on test set: 0.714

The accuracy on the training set is 100%, while the test set accuracy is much worse.

This is an indicative that the tree is overfitting and not generalizing well to new data. 

Therefore, we need to apply pre-pruning to the tree.

We set max_depth=3, limiting the depth of the tree decreases overfitting. 

This leads to a lower accuracy on the training set, but an improvement on the test set.

In [34]:
tree = DecisionTreeClassifier(max_depth=3, random_state=0)
tree.fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test, y_test)))

Accuracy on training set: 0.773
Accuracy on test set: 0.740

###Feature Importance in Decision Trees

Feature importance shows how important each feature is for the decision a tree makes. It is a number between 0 and 1 for each feature, where 0 means “not used at all” and 1 means “perfectly predicts the target”. The feature importances always sum to 1.

In [36]:
print("Feature importances:\n{}".format(tree.feature_importances_))

Feature importances:
[0.04554275 0.6830362 0. 0. 0. 0.27142106
 0. 0. ]

Glucose is the most important feature

In [38]:
def plot_feature_importances_diabetes(model):
    plt.figure(figsize=(8,6))
    n_features = 8
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), diabetes_features)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    plt.ylim(-1, n_features)
    plot_feature_importances_diabetes(tree)
plt.savefig('feature_importance') # I do not know where it is saved

# Random Forest

In [40]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=0) # we use a random forest consisting of 100 trees
rf.fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(rf.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(rf.score(X_test, y_test)))

Accuracy on training set: 1.000
Accuracy on test set: 0.786

The random forest gives us an accuracy of 78.6%, better than the logistic regression model or a single decision tree, without tuning any parameters. However, we can adjust the max_features setting, to see whether the result can be improved.

In [42]:
rf1 = RandomForestClassifier(max_depth=3, n_estimators=100, random_state=0)
rf1.fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(rf1.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(rf1.score(X_test, y_test)))

Accuracy on training set: 0.800
Accuracy on test set: 0.755

After changing **max_depth =3**, the accuracy does not change alot. Thus, this model works well for default parameters.

# Gradient Boosting

In [45]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=0)
gb.fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(gb.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(gb.score(X_test, y_test)))

Accuracy on training set: 0.917
Accuracy on test set: 0.792

It seems that it is overfitting. To reduce overfitting, we could either apply stronger pre-pruning by limiting the maximum depth or lower the learning rate:

In [47]:
gb1 = GradientBoostingClassifier(random_state=0, max_depth=1)
gb1.fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(gb1.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(gb1.score(X_test, y_test)))

Accuracy on training set: 0.804
Accuracy on test set: 0.781

In [48]:
gb2 = GradientBoostingClassifier(random_state=0, learning_rate=0.01)
gb2.fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(gb2.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(gb2.score(X_test, y_test)))

Accuracy on training set: 0.802
Accuracy on test set: 0.776

Both methods of decreasing the model complexity reduced the training set accuracy, as expected. However, in this case, none of these methods increased the generalization performance of the test set.

# Support Vector Machine

In [51]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)
print("Accuracy on training set: {:.2f}".format(svc.score(X_train, y_train)))
print("Accuracy on test set: {:.2f}".format(svc.score(X_test, y_test)))

/databricks/python/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
 "avoid this warning.", FutureWarning)
Accuracy on training set: 1.00
Accuracy on test set: 0.65

The model overfits quite substantially, with a perfect score on the training set and only 65% accuracy on the test set.

SVM requires all the features to vary on a similar scale. We will need to re-scale our data that all the features are approximately on the same scale:

In [53]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

svc = SVC()
svc.fit(X_train_scaled, y_train)

print("Accuracy on training set: {:.2f}".format(svc.score(X_train_scaled, y_train)))
print("Accuracy on test set: {:.2f}".format(svc.score(X_test_scaled, y_test)))

/databricks/python/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
 return self.partial_fit(X, y)
/databricks/python/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
 return self.partial_fit(X, y)
/databricks/python/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
 "avoid this warning.", FutureWarning)
Accuracy on training set: 0.77
Accuracy on test set: 0.77

Scaling the data gives a huge difference! Now we are actually underfitting, where training and test set performance are quite similar but less close to 100% accuracy. 

From here, we can try increasing either C or gamma to fit a more complex model.

In [55]:
svc = SVC(C=1000)
svc.fit(X_train_scaled, y_train)
print("Accuracy on training set: {:.3f}".format(svc.score(X_train_scaled, y_train)))
print("Accuracy on test set: {:.3f}".format(svc.score(X_test_scaled, y_test)))

/databricks/python/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
 "avoid this warning.", FutureWarning)
Accuracy on training set: 0.790
Accuracy on test set: 0.797

We get that the accuracy is improved by changing C

# k-Nearest Neighbors

The k-NN algorithm is arguably the simplest machine learning algorithm. Building the model consists only of storing the training data set. To make a prediction for a new data point, the algorithm finds the closest data points in the training data set — its “nearest neighbors.”

In [58]:
from sklearn.neighbors import KNeighborsClassifier
training_accuracy = []
test_accuracy = []
# try n_neighbors from 1 to 10
neighbors_settings = range(1, 11)
for n_neighbors in neighbors_settings:
    # build the model
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train, y_train)
    # record training set accuracy
    training_accuracy.append(knn.score(X_train, y_train))
    # record test set accuracy
    test_accuracy.append(knn.score(X_test, y_test))
plt.plot(neighbors_settings, training_accuracy, label="training accuracy")
plt.plot(neighbors_settings, test_accuracy, label="test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("n_neighbors")
plt.legend()
plt.savefig('knn_compare_model')

In [59]:
print("the accuracy for training data: {:.3f}".format(knn.score(X_train, y_train)))
print("the accuracy for test data: {:.3f}".format(knn.score(X_test, y_test)))

the accuracy for training data: 0.774
the accuracy for test data: 0.771

The above plot shows the training and test set accuracy on the y-axis against the setting of n_neighbors on the x-axis. Considering if we choose one single nearest neighbor, the prediction on the training set is perfect. But when more neighbors are considered, the training accuracy drops, indicating that using the single nearest neighbor leads to a model that is too complex. The best performance is somewhere around 9 neighbors.

In [61]:
knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(X_train, y_train)
print('Accuracy of K-NN classifier on training set: {:.2f}'.format(knn.score(X_train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'.format(knn.score(X_test, y_test)))

Accuracy of K-NN classifier on training set: 0.79
Accuracy of K-NN classifier on test set: 0.78

#Deep Learning

In [63]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=42)
mlp.fit(X_train, y_train)
print("Accuracy on training set: {:.2f}".format(mlp.score(X_train, y_train)))
print("Accuracy on test set: {:.2f}".format(mlp.score(X_test, y_test)))

Accuracy on training set: 0.73
Accuracy on test set: 0.72

The accuracy of the Multilayer perceptrons (MLP) is not as good as the other models in ML, this is likely due to scaling of the data. Deep learning algorithms also expect all input features to vary in a similar way, and ideally to have a mean of 0, and a variance of 1. We must re-scale our data so that it fulfills these requirements.

In [65]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

mlp = MLPClassifier(random_state=0)
mlp.fit(X_train_scaled, y_train)

print("Accuracy on training set: {:.3f}".format(mlp.score(X_train_scaled, y_train)))
print("Accuracy on test set: {:.3f}".format(mlp.score(X_test_scaled, y_test)))

/databricks/python/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
 return self.partial_fit(X, y)
/databricks/python/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
 return self.fit(X, **fit_params).transform(X)
/databricks/python/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
 return self.partial_fit(X, y)
/databricks/python/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
 return self.fit(X, **fit_params).transform(X)
/databricks/python/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
 % self.max_iter, ConvergenceWarning)
Accuracy on training set: 0.823
Accuracy on test set: 0.802

Now try to increase the number of iterations

In [67]:
mlp = MLPClassifier(max_iter=1000, random_state=0)
mlp.fit(X_train_scaled, y_train)
print("Accuracy on training set: {:.3f}".format(mlp.score(X_train_scaled, y_train)))
print("Accuracy on test set: {:.3f}".format(mlp.score(X_test_scaled, y_test)))

/databricks/python/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
 % self.max_iter, ConvergenceWarning)
Accuracy on training set: 0.908
Accuracy on test set: 0.792

Increasing the number of iterations only increased the training set performance, not the test set performance.

Let’s increase the alpha parameter and add stronger regularization of the weights:

In [69]:
mlp = MLPClassifier(max_iter=1000, alpha=1, random_state=0)
mlp.fit(X_train_scaled, y_train)
print("Accuracy on training set: {:.3f}".format(mlp.score(X_train_scaled, y_train)))
print("Accuracy on test set: {:.3f}".format(mlp.score(X_test_scaled, y_test)))

Accuracy on training set: 0.806
Accuracy on test set: 0.797

The result sounds better, but we are not able to increase the test accuracy further.

Therefore, our best model so far is default deep learning model after scaling.

## Conclusion
We use many ML algorithm of classication and regression to analyse our data. This tells us how to control the complexity of each. 
And which algorithm will be better for these data.